In [1]:
%pip install -U transformers
%pip install -U datasets
%pip install -U accelerate
%pip install -U peft
%pip install -U trl
%pip install -U bitsandbytes
%pip install -U wandb

In [2]:
!pip install evaluate huggingface_hub

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset
import evaluate
import torch

# Step 1: Load a smaller model and tokenizer
model_name = "bert-base-uncased"  # Use smaller models like distilbert or albert
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Adjust num_labels for task

# Step 2: Load the GLUE dataset
glue_task = "cola"  # Replace with the desired GLUE task
dataset = load_dataset("glue", glue_task)

# Step 3: Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(
        examples["sentence"],  # Replace "sentence" with the correct field for the task
        padding="max_length",
        truncation=True,
        max_length=128,  # Reduce max length for smaller memory usage
    )

encoded_dataset = dataset.map(tokenize_function, batched=True)

# Add the labels to the dataset
encoded_dataset = encoded_dataset.map(
    lambda examples: {
        "input_ids": examples["input_ids"],
        "attention_mask": examples["attention_mask"],
        "labels": torch.tensor(examples["label"], dtype=torch.long),
    },
    batched=True,
)

# Set format for PyTorch
encoded_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# Step 4: Define Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=3,  # Smaller models can handle larger batch sizes
    num_train_epochs=1,
    weight_decay=0.01,
    fp16=True,  # Mixed precision training
    logging_dir="./logs",
    save_strategy="epoch",
    load_best_model_at_end=True,  # Load the best model during evaluation
    metric_for_best_model="accuracy",  # Focus on accuracy as the primary metric
)

# Step 5: Define the accuracy metric
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

# Step 6: Initialize Trainer with compute_metrics
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Step 7: Train the Model
trainer.train()

# Step 8: Evaluate the Model
results = trainer.evaluate()
print(f"Evaluation results: {results}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1063 [00:00<?, ? examples/s]

Map:   0%|          | 0/1063 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-6-2a30dfe53eac>:63: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.656100,0.686191,0.815916


Evaluation results: {'eval_loss': 0.6861907839775085, 'eval_accuracy': 0.8159156279961649, 'eval_runtime': 2.962, 'eval_samples_per_second': 352.131, 'eval_steps_per_second': 44.227, 'epoch': 1.0}


In [3]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"  # Enable more detailed error messages

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset
import evaluate
import torch

# Step 1: Load the model and tokenizer
model_name = "bert-base-uncased"  # You can replace this with any other smaller model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)  # MNLI has 3 labels

# Step 2: Load the GLUE MNLI dataset
dataset = load_dataset("glue", "mnli")

# Step 3: Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(
        examples["premise"], examples["hypothesis"],  # MNLI has two input fields
        padding="max_length",
        truncation=True,
        max_length=128,
    )

encoded_dataset = dataset.map(tokenize_function, batched=True)

# Add the labels to the dataset
encoded_dataset = encoded_dataset.map(
    lambda examples: {
        "input_ids": examples["input_ids"],
        "attention_mask": examples["attention_mask"],
        "labels": torch.tensor(examples["label"], dtype=torch.long),
    },
    batched=True,
)

# Set format for PyTorch
encoded_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# Step 4: Define Training Arguments
training_args = TrainingArguments(
    output_dir="./results_mnli",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # Adjust batch size based on available memory
    num_train_epochs=1,
    weight_decay=0.01,
    fp16=True,  # Mixed precision training
    logging_dir="./logs_mnli",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

# Step 5: Define the accuracy metric
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

# Step 6: Initialize Trainer with compute_metrics
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation_matched"],  # Use the "matched" validation set for MNLI
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Step 7: Train the Model
trainer.train()

# Step 8: Evaluate the Model
results = trainer.evaluate()
print(f"Evaluation results: {results}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/9832 [00:00<?, ? examples/s]

Map:   0%|          | 0/9832 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-5-60e81beeb074>:62: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: rakibulislamprince10 (rakibulislamprince10-purdue-university). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,0.485400,0.454289,0.835252


Evaluation results: {'eval_loss': 0.4542885720729828, 'eval_accuracy': 0.8352521650534895, 'eval_runtime': 27.107, 'eval_samples_per_second': 362.084, 'eval_steps_per_second': 45.265, 'epoch': 1.0}
